# Ajuste de Tipos de Datos en PostgreSQL

Este notebook tiene como objetivo verificar y ajustar los tipos de datos de las columnas en PostgreSQL después de la carga inicial desde archivos CSV.

## Objetivo

Cuando se cargan datos desde CSV usando `COPY` sin especificar tipos de datos explícitos en pandas, PostgreSQL puede inferir tipos incorrectos o almacenar datos como `VARCHAR`/`TEXT` cuando deberían ser tipos específicos como `NUMERIC`, `TIMESTAMP` o `INTEGER`.

Este notebook:
1. Carga cada tabla en un DataFrame de pandas
2. Verifica los tipos de datos actuales usando `.info()`
3. Identifica discrepancias entre los tipos actuales y los esperados
4. Ejecuta los ajustes necesarios usando SQL `ALTER TABLE` cuando sea requerido


## Tabla de Verificación de Tipos de Datos

A continuación se presenta la lista completa de las **16 columnas** que requieren verificación y ajuste de tipos de datos, agrupadas por tabla:


| Tabla | Columna | Tipo de Dato Esperado | Descripción |
|-------|---------|----------------------|-------------|
| **usuarios** | `fecha_registro` | TIMESTAMP | Fecha de registro del usuario |
| **productos** | `precio` | NUMERIC(10, 2) | Precio unitario del producto |
| **productos** | `stock` | INTEGER | Cantidad disponible en inventario |
| **ordenes** | `fecha_orden` | TIMESTAMP | Fecha y hora de creación de la orden |
| **ordenes** | `total` | NUMERIC(10, 2) | Monto total de la orden |
| **ordenes** | `estado` | ENUM | Estado de la orden (Pendiente, Enviado, Completado, Cancelado) |
| **detalle_ordenes** | `precio_unitario` | NUMERIC(10, 2) | Precio unitario al momento de la orden |
| **detalle_ordenes** | `cantidad` | INTEGER | Cantidad de productos en el detalle |
| **carrito** | `fecha_agregado` | TIMESTAMP | Fecha en que se agregó el producto al carrito |
| **carrito** | `cantidad` | INTEGER | Cantidad del producto en el carrito |
| **ordenes_metodos_pago** | `monto_pagado` | NUMERIC(10, 2) | Monto pagado con cada método |
| **resenas_productos** | `fecha` | TIMESTAMP | Fecha de publicación de la reseña |
| **resenas_productos** | `calificacion` | INTEGER | Calificación del producto (1-5) |
| **historial_pagos** | `monto` | NUMERIC(10, 2) | Monto del pago |
| **historial_pagos** | `fecha_pago` | TIMESTAMP | Fecha del pago |
| **historial_pagos** | `estado_pago` | ENUM | Estado del pago (Procesando, Pagado, Fallido, Reembolsado) |

**Total: 16 columnas requieren verificación de tipos de datos**


## 1. Importaciones y Configuración Inicial


In [ ]:
# Importar bibliotecas necesarias
import sys
import os

try:
    from ..Utils.path_manager import PathManager
except ImportError:
    # Si falla el import relativo 
    current_dir = os.getcwd()  # Directorio actual de trabajo
    if 'Notebooks' in current_dir:
        avance1_dir = os.path.dirname(current_dir)
    else:
        # Intentar calcular desde la ubicación relativa
        avance1_dir = os.path.abspath(os.path.join(current_dir, '..'))
    
    utils_dir = os.path.join(avance1_dir, 'Utils')
    if utils_dir not in sys.path:
        sys.path.insert(0, utils_dir)
    from path_manager import PathManager

# Configurar sys.path usando PathManager (patrón Singleton)
path_manager = PathManager.get_instance()
path_manager.setup_sys_path()

# Importar DBConnector desde la raíz del proyecto
from Database.db_connector import DBConnector

# Importar bibliotecas de análisis
import pandas as pd
import numpy as np

print("✓ Bibliotecas importadas correctamente")
print(f"✓ Project root configurado: {path_manager.get_project_root()}")


In [ ]:
# Obtener el Engine de la base de datos usando DBConnector (patrón Singleton)
db = DBConnector.get_instance()
engine = db.get_engine()

print("✓ Conexión a la base de datos establecida")
print(f"✓ Engine obtenido: {type(engine).__name__}")


### Función para Cargar Tablas

Función genérica para cargar cualquier tabla de la base de datos en un DataFrame de pandas.


In [ ]:
def load_table_to_df(table_name, engine):
    """
    Carga una tabla completa de la base de datos PostgreSQL en un DataFrame de pandas.
    
    Args:
        table_name (str): Nombre de la tabla a cargar
        engine: Engine de SQLAlchemy para la conexión a la base de datos
    
    Returns:
        pd.DataFrame: DataFrame con todos los datos de la tabla
    
    Example:
        >>> df_usuarios = load_table_to_df('usuarios', engine)
    """
    # Una consulta sencilla para cargar la tabla completa
    query = f"SELECT * FROM {table_name};"
    
    return pd.read_sql(query, engine)


## 2. Verificación de Tipos de Datos por Tabla

A continuación cargamos cada tabla y verificamos los tipos de datos actuales usando `.info()` y comparamos con los tipos esperados.


### 2.1. Tabla: usuarios

**Columnas a verificar:**
- `fecha_registro` → TIMESTAMP


In [ ]:
# Cargar tabla usuarios
df_usuarios = load_table_to_df('usuarios', engine)

print("=" * 80)
print("TABLA: usuarios")
print("=" * 80)
print(f"\nForma del DataFrame: {df_usuarios.shape}")
print(f"\nPrimeras filas:")
print(df_usuarios.head())
print(f"\nInformación de tipos de datos:")
df_usuarios.info()
print(f"\nTipos de datos por columna:")
print(df_usuarios.dtypes)


### 2.2. Tabla: productos

**Columnas a verificar:**
- `precio` → NUMERIC(10, 2)
- `stock` → INTEGER


In [ ]:
# Cargar tabla productos
df_productos = load_table_to_df('productos', engine)

print("=" * 80)
print("TABLA: productos")
print("=" * 80)
print(f"\nForma del DataFrame: {df_productos.shape}")
print(f"\nPrimeras filas:")
print(df_productos.head())
print(f"\nInformación de tipos de datos:")
df_productos.info()
print(f"\nTipos de datos por columna:")
print(df_productos.dtypes)


### 2.3. Tabla: ordenes

**Columnas a verificar:**
- `fecha_orden` → TIMESTAMP
- `total` → NUMERIC(10, 2)
- `estado` → ENUM


In [ ]:
# Cargar tabla ordenes
df_ordenes = load_table_to_df('ordenes', engine)

print("=" * 80)
print("TABLA: ordenes")
print("=" * 80)
print(f"\nForma del DataFrame: {df_ordenes.shape}")
print(f"\nPrimeras filas:")
print(df_ordenes.head())
print(f"\nInformación de tipos de datos:")
df_ordenes.info()
print(f"\nTipos de datos por columna:")
print(df_ordenes.dtypes)


### 2.4. Tabla: detalle_ordenes

**Columnas a verificar:**
- `precio_unitario` → NUMERIC(10, 2)
- `cantidad` → INTEGER


In [ ]:
# Cargar tabla detalle_ordenes
df_detalle_ordenes = load_table_to_df('detalle_ordenes', engine)

print("=" * 80)
print("TABLA: detalle_ordenes")
print("=" * 80)
print(f"\nForma del DataFrame: {df_detalle_ordenes.shape}")
print(f"\nPrimeras filas:")
print(df_detalle_ordenes.head())
print(f"\nInformación de tipos de datos:")
df_detalle_ordenes.info()
print(f"\nTipos de datos por columna:")
print(df_detalle_ordenes.dtypes)


### 2.5. Tabla: carrito

**Columnas a verificar:**
- `fecha_agregado` → TIMESTAMP
- `cantidad` → INTEGER


In [ ]:
# Cargar tabla carrito
df_carrito = load_table_to_df('carrito', engine)

print("=" * 80)
print("TABLA: carrito")
print("=" * 80)
print(f"\nForma del DataFrame: {df_carrito.shape}")
print(f"\nPrimeras filas:")
print(df_carrito.head())
print(f"\nInformación de tipos de datos:")
df_carrito.info()
print(f"\nTipos de datos por columna:")
print(df_carrito.dtypes)


### 2.6. Tabla: ordenes_metodos_pago

**Columnas a verificar:**
- `monto_pagado` → NUMERIC(10, 2)


In [ ]:
# Cargar tabla ordenes_metodos_pago
df_ordenes_metodos_pago = load_table_to_df('ordenes_metodos_pago', engine)

print("=" * 80)
print("TABLA: ordenes_metodos_pago")
print("=" * 80)
print(f"\nForma del DataFrame: {df_ordenes_metodos_pago.shape}")
print(f"\nPrimeras filas:")
print(df_ordenes_metodos_pago.head())
print(f"\nInformación de tipos de datos:")
df_ordenes_metodos_pago.info()
print(f"\nTipos de datos por columna:")
print(df_ordenes_metodos_pago.dtypes)


### 2.7. Tabla: resenas_productos

**Columnas a verificar:**
- `fecha` → TIMESTAMP
- `calificacion` → INTEGER


In [ ]:
# Cargar tabla resenas_productos
df_resenas_productos = load_table_to_df('resenas_productos', engine)

print("=" * 80)
print("TABLA: resenas_productos")
print("=" * 80)
print(f"\nForma del DataFrame: {df_resenas_productos.shape}")
print(f"\nPrimeras filas:")
print(df_resenas_productos.head())
print(f"\nInformación de tipos de datos:")
df_resenas_productos.info()
print(f"\nTipos de datos por columna:")
print(df_resenas_productos.dtypes)


### 2.8. Tabla: historial_pagos

**Columnas a verificar:**
- `monto` → NUMERIC(10, 2)
- `fecha_pago` → TIMESTAMP
- `estado_pago` → ENUM


In [ ]:
# Cargar tabla historial_pagos
df_historial_pagos = load_table_to_df('historial_pagos', engine)

print("=" * 80)
print("TABLA: historial_pagos")
print("=" * 80)
print(f"\nForma del DataFrame: {df_historial_pagos.shape}")
print(f"\nPrimeras filas:")
print(df_historial_pagos.head())
print(f"\nInformación de tipos de datos:")
df_historial_pagos.info()
print(f"\nTipos de datos por columna:")
print(df_historial_pagos.dtypes)


## 3. Verificación de Tipos de Datos en PostgreSQL

A continuación verificamos los tipos de datos actuales directamente desde PostgreSQL usando `information_schema`:


In [ ]:
# Consulta para verificar tipos de datos actuales en PostgreSQL
query_tipos = """
SELECT 
    table_name, 
    column_name, 
    data_type, 
    numeric_precision, 
    numeric_scale,
    udt_name
FROM information_schema.columns
WHERE table_schema = 'public'
    AND column_name IN (
        'fecha_registro', 'precio', 'stock', 'fecha_orden', 'total', 'estado',
        'precio_unitario', 'cantidad', 'fecha_agregado', 'monto_pagado',
        'fecha', 'calificacion', 'monto', 'fecha_pago', 'estado_pago'
    )
ORDER BY table_name, column_name;
"""

df_tipos_actuales = pd.read_sql(query_tipos, engine)

print("=" * 80)
print("TIPOS DE DATOS ACTUALES EN POSTGRESQL")
print("=" * 80)
print(df_tipos_actuales.to_string(index=False))


## 4. Comparación de Tipos Esperados vs Actuales

A continuación creamos una tabla comparativa para identificar discrepancias:


In [ ]:
# Definir tipos esperados
tipos_esperados = {
    ('usuarios', 'fecha_registro'): 'TIMESTAMP',
    ('productos', 'precio'): 'NUMERIC(10, 2)',
    ('productos', 'stock'): 'INTEGER',
    ('ordenes', 'fecha_orden'): 'TIMESTAMP',
    ('ordenes', 'total'): 'NUMERIC(10, 2)',
    ('ordenes', 'estado'): 'ENUM',
    ('detalle_ordenes', 'precio_unitario'): 'NUMERIC(10, 2)',
    ('detalle_ordenes', 'cantidad'): 'INTEGER',
    ('carrito', 'fecha_agregado'): 'TIMESTAMP',
    ('carrito', 'cantidad'): 'INTEGER',
    ('ordenes_metodos_pago', 'monto_pagado'): 'NUMERIC(10, 2)',
    ('resenas_productos', 'fecha'): 'TIMESTAMP',
    ('resenas_productos', 'calificacion'): 'INTEGER',
    ('historial_pagos', 'monto'): 'NUMERIC(10, 2)',
    ('historial_pagos', 'fecha_pago'): 'TIMESTAMP',
    ('historial_pagos', 'estado_pago'): 'ENUM'
}

# Crear DataFrame comparativo
comparacion = []
for (table, column), tipo_esperado in tipos_esperados.items():
    # Buscar el tipo actual en df_tipos_actuales
    tipo_actual = df_tipos_actuales[
        (df_tipos_actuales['table_name'] == table) & 
        (df_tipos_actuales['column_name'] == column)
    ]
    
    if not tipo_actual.empty:
        data_type = tipo_actual.iloc[0]['data_type']
        udt_name = tipo_actual.iloc[0]['udt_name']
        numeric_precision = tipo_actual.iloc[0]['numeric_precision']
        numeric_scale = tipo_actual.iloc[0]['numeric_scale']
        
        # Formatear tipo actual para comparación
        if data_type == 'numeric':
            tipo_actual_str = f"NUMERIC({numeric_precision}, {numeric_scale})"
        elif data_type == 'timestamp without time zone':
            tipo_actual_str = 'TIMESTAMP'
        elif data_type == 'integer':
            tipo_actual_str = 'INTEGER'
        elif data_type == 'USER-DEFINED' and 'enum' in udt_name.lower():
            tipo_actual_str = 'ENUM'
        else:
            tipo_actual_str = f"{data_type} ({udt_name})"
        
        necesita_ajuste = tipo_actual_str != tipo_esperado
        
        comparacion.append({
            'Tabla': table,
            'Columna': column,
            'Tipo Esperado': tipo_esperado,
            'Tipo Actual': tipo_actual_str,
            'Necesita Ajuste': 'SÍ' if necesita_ajuste else 'NO'
        })
    else:
        comparacion.append({
            'Tabla': table,
            'Columna': column,
            'Tipo Esperado': tipo_esperado,
            'Tipo Actual': 'NO ENCONTRADO',
            'Necesita Ajuste': 'SÍ'
        })

df_comparacion = pd.DataFrame(comparacion)

print("=" * 80)
print("COMPARACIÓN: TIPOS ESPERADOS vs ACTUALES")
print("=" * 80)
print(df_comparacion.to_string(index=False))

print("\n" + "=" * 80)
print("RESUMEN DE AJUSTES NECESARIOS")
print("=" * 80)
ajustes_necesarios = df_comparacion[df_comparacion['Necesita Ajuste'] == 'SÍ']
print(f"\nTotal de columnas que requieren ajuste: {len(ajustes_necesarios)}")
if len(ajustes_necesarios) > 0:
    print("\nColumnas que requieren ajuste:")
    print(ajustes_necesarios[['Tabla', 'Columna', 'Tipo Esperado', 'Tipo Actual']].to_string(index=False))


### 5.1. Función para Ejecutar Ajustes SQL

Función auxiliar para ejecutar comandos SQL de ajuste de tipos:


In [ ]:
from sqlalchemy import text

def ejecutar_ajuste_sql(sql_command, description):
    """
    Ejecuta un comando SQL de ajuste de tipo de datos.
    
    Args:
        sql_command (str): Comando SQL a ejecutar
        description (str): Descripción del ajuste que se está realizando
    
    Returns:
        bool: True si se ejecutó correctamente, False en caso contrario
    """
    try:
        with engine.connect() as conn:
            conn.execute(text(sql_command))
            conn.commit()
        print(f"✓ {description}")
        return True
    except Exception as e:
        print(f"✗ Error en {description}: {str(e)}")
        return False


### 5.2. Ajuste de Columnas NUMERIC(10, 2)

**IMPORTANTE**: Antes de ejecutar estos ALTER TABLE, asegúrate de que los datos en las columnas sean convertibles a NUMERIC. Si hay valores con símbolos de moneda ($, €), comas como separadores de miles, o espacios, necesitarás limpiarlos primero.


In [ ]:
# Ajustar columnas NUMERIC(10, 2)
ajustes_numeric = [
    ("ALTER TABLE productos ALTER COLUMN precio TYPE NUMERIC(10, 2) USING precio::NUMERIC(10, 2);", 
     "productos.precio → NUMERIC(10, 2)"),
    ("ALTER TABLE ordenes ALTER COLUMN total TYPE NUMERIC(10, 2) USING total::NUMERIC(10, 2);", 
     "ordenes.total → NUMERIC(10, 2)"),
    ("ALTER TABLE detalle_ordenes ALTER COLUMN precio_unitario TYPE NUMERIC(10, 2) USING precio_unitario::NUMERIC(10, 2);", 
     "detalle_ordenes.precio_unitario → NUMERIC(10, 2)"),
    ("ALTER TABLE ordenes_metodos_pago ALTER COLUMN monto_pagado TYPE NUMERIC(10, 2) USING monto_pagado::NUMERIC(10, 2);", 
     "ordenes_metodos_pago.monto_pagado → NUMERIC(10, 2)"),
    ("ALTER TABLE historial_pagos ALTER COLUMN monto TYPE NUMERIC(10, 2) USING monto::NUMERIC(10, 2);", 
     "historial_pagos.monto → NUMERIC(10, 2)")
]

print("=" * 80)
print("AJUSTANDO COLUMNAS NUMERIC(10, 2)")
print("=" * 80)

# Descomentar las siguientes líneas para ejecutar los ajustes:
# for sql, desc in ajustes_numeric:
#     ejecutar_ajuste_sql(sql, desc)


### 5.3. Ajuste de Columnas TIMESTAMP

**IMPORTANTE**: Antes de ejecutar estos ALTER TABLE, asegúrate de que los datos en las columnas de fecha estén en un formato que PostgreSQL pueda interpretar. El formato esperado es `YYYY-MM-DD HH:MM:SS` o variaciones compatibles.


In [ ]:
# Ajustar columnas TIMESTAMP
ajustes_timestamp = [
    ("ALTER TABLE usuarios ALTER COLUMN fecha_registro TYPE TIMESTAMP USING fecha_registro::TIMESTAMP;", 
     "usuarios.fecha_registro → TIMESTAMP"),
    ("ALTER TABLE ordenes ALTER COLUMN fecha_orden TYPE TIMESTAMP USING fecha_orden::TIMESTAMP;", 
     "ordenes.fecha_orden → TIMESTAMP"),
    ("ALTER TABLE carrito ALTER COLUMN fecha_agregado TYPE TIMESTAMP USING fecha_agregado::TIMESTAMP;", 
     "carrito.fecha_agregado → TIMESTAMP"),
    ("ALTER TABLE resenas_productos ALTER COLUMN fecha TYPE TIMESTAMP USING fecha::TIMESTAMP;", 
     "resenas_productos.fecha → TIMESTAMP"),
    ("ALTER TABLE historial_pagos ALTER COLUMN fecha_pago TYPE TIMESTAMP USING fecha_pago::TIMESTAMP;", 
     "historial_pagos.fecha_pago → TIMESTAMP")
]

print("=" * 80)
print("AJUSTANDO COLUMNAS TIMESTAMP")
print("=" * 80)

# Descomentar las siguientes líneas para ejecutar los ajustes:
# for sql, desc in ajustes_timestamp:
#     ejecutar_ajuste_sql(sql, desc)


### 5.4. Ajuste de Columnas INTEGER

**IMPORTANTE**: Antes de ejecutar estos ALTER TABLE, asegúrate de que los datos sean números enteros válidos sin decimales, espacios o caracteres especiales.


In [ ]:
# Ajustar columnas INTEGER
ajustes_integer = [
    ("ALTER TABLE productos ALTER COLUMN stock TYPE INTEGER USING stock::INTEGER;", 
     "productos.stock → INTEGER"),
    ("ALTER TABLE detalle_ordenes ALTER COLUMN cantidad TYPE INTEGER USING cantidad::INTEGER;", 
     "detalle_ordenes.cantidad → INTEGER"),
    ("ALTER TABLE carrito ALTER COLUMN cantidad TYPE INTEGER USING cantidad::INTEGER;", 
     "carrito.cantidad → INTEGER"),
    ("ALTER TABLE resenas_productos ALTER COLUMN calificacion TYPE INTEGER USING calificacion::INTEGER;", 
     "resenas_productos.calificacion → INTEGER")
]

print("=" * 80)
print("AJUSTANDO COLUMNAS INTEGER")
print("=" * 80)

# Descomentar las siguientes líneas para ejecutar los ajustes:
# for sql, desc in ajustes_integer:
#     ejecutar_ajuste_sql(sql, desc)


### 5.5. Validación de Columnas ENUM

**IMPORTANTE**: Los ENUMs se crean correctamente con SQLAlchemy, pero es importante validar que los valores en el CSV coincidan exactamente con los valores permitidos del enum. Si hay discrepancias, necesitarás limpiar los datos antes de la carga o usar una función de conversión.


In [ ]:
# Validar valores ENUM
print("=" * 80)
print("VALIDACIÓN DE VALORES ENUM")
print("=" * 80)

# Verificar valores únicos en ordenes.estado
query_estado_orden = """
SELECT DISTINCT estado, COUNT(*) as cantidad
FROM ordenes
GROUP BY estado
ORDER BY cantidad DESC;
"""
df_estado_orden = pd.read_sql(query_estado_orden, engine)
print("\nValores únicos en ordenes.estado:")
print(df_estado_orden.to_string(index=False))

# Verificar valores únicos en historial_pagos.estado_pago
query_estado_pago = """
SELECT DISTINCT estado_pago, COUNT(*) as cantidad
FROM historial_pagos
GROUP BY estado_pago
ORDER BY cantidad DESC;
"""
df_estado_pago = pd.read_sql(query_estado_pago, engine)
print("\nValores únicos en historial_pagos.estado_pago:")
print(df_estado_pago.to_string(index=False))

# Valores permitidos
valores_permitidos_estado = ['Pendiente', 'Enviado', 'Completado', 'Cancelado']
valores_permitidos_pago = ['Procesando', 'Pagado', 'Fallido', 'Reembolsado']

print(f"\nValores permitidos para ordenes.estado: {valores_permitidos_estado}")
print(f"Valores permitidos para historial_pagos.estado_pago: {valores_permitidos_pago}")

# Verificar si hay valores no permitidos
valores_orden = set(df_estado_orden['estado'].unique())
valores_pago = set(df_estado_pago['estado_pago'].unique())

valores_invalidos_orden = valores_orden - set(valores_permitidos_estado)
valores_invalidos_pago = valores_pago - set(valores_permitidos_pago)

if valores_invalidos_orden:
    print(f"\n⚠ ADVERTENCIA: Valores no permitidos en ordenes.estado: {valores_invalidos_orden}")
else:
    print(f"\n✓ Todos los valores en ordenes.estado son válidos")

if valores_invalidos_pago:
    print(f"⚠ ADVERTENCIA: Valores no permitidos en historial_pagos.estado_pago: {valores_invalidos_pago}")
else:
    print(f"✓ Todos los valores en historial_pagos.estado_pago son válidos")


## 6. Verificación Final

Después de ejecutar los ajustes, verifica nuevamente los tipos de datos para confirmar que se aplicaron correctamente:


In [ ]:
# Verificación final de tipos de datos
query_verificacion_final = """
SELECT 
    table_name, 
    column_name, 
    data_type, 
    numeric_precision, 
    numeric_scale,
    udt_name
FROM information_schema.columns
WHERE table_schema = 'public'
    AND column_name IN (
        'fecha_registro', 'precio', 'stock', 'fecha_orden', 'total', 'estado',
        'precio_unitario', 'cantidad', 'fecha_agregado', 'monto_pagado',
        'fecha', 'calificacion', 'monto', 'fecha_pago', 'estado_pago'
    )
ORDER BY table_name, column_name;
"""

df_verificacion_final = pd.read_sql(query_verificacion_final, engine)

print("=" * 80)
print("VERIFICACIÓN FINAL DE TIPOS DE DATOS")
print("=" * 80)
print(df_verificacion_final.to_string(index=False))
